In [1]:
# data preprocessing
import pandas as pd
import numpy as np

# data visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from wordcloud import WordCloud

# sklearn 
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

import re
import ast # 형변환 

from pprint import pprint as pp
import warnings
warnings.filterwarnings(action='ignore')

# 텍스트 마이닝
# from eunjeon import Mecab
# import nltk
# from konlpy.tag import Okt
# from nltk import FreqDist
# from gensim.models import Word2Vec

In [2]:
# font
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
matplotlib.rcParams['axes.unicode_minus'] = False   

# 1. 데이터 불러오기

In [201]:
# import data
import pandas as pd
df_raw = pd.read_csv("df_animal")

# 활용 데이터 프레임화 
df = df_raw.copy()
df.drop('Unnamed: 0', axis = 1, inplace = True)
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324702 entries, 0 to 324701
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   desertionno   324702 non-null  int64  
 1   popfile       324702 non-null  object 
 2   happendt      324702 non-null  int64  
 3   happenplace   324702 non-null  object 
 4   kindcd        324702 non-null  object 
 5   colorcd       324702 non-null  object 
 6   age           324702 non-null  object 
 7   weight        324702 non-null  object 
 8   noticeno      324701 non-null  object 
 9   noticesdt     324702 non-null  int64  
 10  noticeedt     324702 non-null  int64  
 11  sexcd         324702 non-null  object 
 12  neuteryn      324702 non-null  object 
 13  specialmark   324702 non-null  object 
 14  carenm        324702 non-null  object 
 15  orgnm         324702 non-null  object 
 16  processstate  324702 non-null  object 
 17  type          324702 non-null  object 
 18  lat 

In [4]:
df.head()

,desertionno,popfile,happendt,happenplace,kindcd,colorcd,age,weight,noticeno,noticesdt,noticeedt,sexcd,neuteryn,specialmark,carenm,orgnm,processstate,type,lat,long
0,448535202001458,http://www.animal.go.kr/files/shelter/2020/07/...,20201013,김해시 주촌면 서부로1563번길 54,[개] 믹스견,검흰,2016(년생),8(Kg),경남-김해-2020-01466,20201015,20201026,F,U,1469,유기동물및관리협회,경상남도 김해시,보호중,dog,35.228545,128.889352
1,446500202000046,http://www.animal.go.kr/files/shelter/2020/10/...,20201013,군내면 송산길 118,[개] 믹스견,황색,2019(년생),7(Kg),전남-진도-2020-00046,20201015,20201022,F,N,매우 사나움,진도개메디컬센터,전라남도 진도군,보호중,dog,34.486871,126.263485
2,446500202000045,http://www.animal.go.kr/files/shelter/2020/10/...,20201013,군내면 송산길 118,[개] 믹스견,백색,2020(년생),10(Kg),전남-진도-2020-00045,20201015,20201022,M,N,매우 사나움,진도개메디컬센터,전라남도 진도군,보호중,dog,34.486871,126.263485
3,446500202000044,http://www.animal.go.kr/files/shelter/2020/10/...,20201013,고군면 가계길 26-10,[개] 믹스견,황색,2019(년생),15(Kg),전남-진도-2020-00044,20201015,20201022,F,N,사나움,진도개메디컬센터,전라남도 진도군,보호중,dog,34.486871,126.263485
4,446499202000215,http://www.animal.go.kr/files/shelter/2020/07/...,20201013,금일 월송리 마을야산,[개] 믹스견,흰색,2017(년생),14(Kg),전남-완도-2020-00218,20201015,20201024,F,N,"귀쪽 갈색,온순한편",유기동물임시보호센터,전라남도 완도군,보호중,dog,34.311060,126.755054


# 1. 불필요한 변수 제거

In [202]:
# 이미지 활용을 하지 않으므로 이미지 관련 변수 제거
df.drop('popfile', axis = 1, inplace = True)

# 2. 결측치 확인

In [203]:
df.noticeno.isnull().value_counts()

False    324701
True          1
Name: noticeno, dtype: int64

In [204]:
df[df['noticeno'].isnull() == True]

,desertionno,happendt,happenplace,kindcd,colorcd,age,weight,noticeno,noticesdt,noticeedt,sexcd,neuteryn,specialmark,carenm,orgnm,processstate,type,lat,long
134828,448536201901146,20190612,삼랑진,[개] 믹스견,갈색,2018(년생),5(Kg),NaN,20190613,20190624,F,U,온순함,밀양시동물보호소,경상남도 밀양시,종료(입양),dog,35.50376,128.746442


# 3. 이상치 확인 및 처리

## 날짜 데이터 이상치 확인 및 데이터 변경 

In [205]:
df.iloc[231419, :] #2170918 --> '2017098' 
df.noticeedt[231419] = '20170918'

df.iloc[231489, :] #2170921 --> '20170921' 
df.noticeedt[231489] = '20170921' 

In [206]:
df.age.unique()

array(['2016(년생)', '2019(년생)', '2020(년생)', '2017(년생)', '2013(년생)',
       '2018(년생)', '2012(년생)', '2015(년생)', '2014(년생)', '2011(년생)',
       '2005(년생)', '2010(년생)', '2008(년생)', '2007(년생)', '2006(년생)',
       '2003(년생)', '2002(년생)', '2009(년생)', '2004(년생)', '1999(년생)',
       '2000(년생)', '2001(년생)', '4(년생)', '2(년생)', '20160409(년생)',
       '20200407(년생)', '20200110(년생)', '1998(년생)', '1996(년생)', '0(년생)',
       '3(년생)', '1997(년생)', '219(년생)', '1995(년생)', '1991(년생)', '1(년생)',
       '5(년생)', '1988(년생)', ' (년생)', '10(년생)', '2015 (년생)', '-(년생)',
       '1992(년생)', '`(년생)'], dtype=object)

In [207]:
df.age[z] = np.nan

In [208]:
df['age'].unique()

array(['2016(년생)', '2019(년생)', '2020(년생)', '2017(년생)', '2013(년생)',
       '2018(년생)', '2012(년생)', '2015(년생)', '2014(년생)', '2011(년생)',
       '2005(년생)', '2010(년생)', '2008(년생)', '2007(년생)', '2006(년생)',
       '2003(년생)', '2002(년생)', '2009(년생)', '2004(년생)', '1999(년생)',
       '2000(년생)', '2001(년생)', nan, '1998(년생)', '1996(년생)', '1997(년생)',
       '1995(년생)', '1991(년생)', '1988(년생)', '1992(년생)'], dtype=object)

# 4. 데이터 형변환

## 나이 변환
### 문자열 제거 및 int형으로 변환

In [209]:
def age_in_year(born_year):
    p = re.compile('\d{4}\(\년\생\)')
    m = p.match(born_year)
    if m:
        return(born_year)
    else:
        return('0')

q = df.age
w = list(q.map(age_in_year))
e = w.count('0')

# e = list(df.age.map(age_in_year)).count('0')

z = []
for i in range(0, len(w)):
    if w[i] == '0':
        z.append(i)

# df = df.drop(z) # 행 삭제

df['age'] = df['age'].map(lambda x: x[0:4])

TypeError: expected string or bytes-like object

In [ ]:
# 나이 float형으로 변환
df.age = df['age'].astype('float')

In [ ]:
# 나이 계산하기 - 태어난 년도를 n살로 바꾸기
# 태어난 연도 - 공지 기준 연도 + 1

df['age'] = pd.DatetimeIndex(df.noticesdt).year - df.age+1

In [ ]:
null_list = df[df.age.isna() == True].index #결측 처리 데이터 9999, 개수: 58개
len(null_list)

In [ ]:
df.age[df.age < 0 ] = np.nan
# df.age.iloc[null_list] = np.nan

## 날짜 변환

### datetime 타입으로 변환 / 년월일 칼럼 별도 생성

In [210]:
# 날짜 타입으로 바꾸기 

def int_to_date(col_name):
    col_name = col_name.astype(str)
    col_name = col_name.str[0:4] + "/" + col_name.str[4:6] + "/" + col_name.str[6:8] 
    col_name = col_name.astype('datetime64[ns]')
    
    return col_name

df.happendt = int_to_date(df.happendt)
df.noticesdt = int_to_date(df.noticesdt)
df.noticeedt = int_to_date(df.noticeedt)


In [211]:
# 날짜 년/월/일 각 컬럼 생성
df['noticesdt_year'] = df.noticesdt.dt.year
df['noticesdt_month'] = df.noticesdt.dt.month
df['noticesdt_day'] = df.noticesdt.dt.day

## 무게 변환

In [212]:
# 체중 : 체중 표기 방법이 아닌경우, nan 리턴 


def weight_reg_01(x):
    
    x = str(x)
    idx = x.find('(') 
    x = x[:idx] 
    
    m = re.match('\d+\,+|\d+\;+|\d+\:+|\d+\/+|\d+\-+|\d+\~+', x) # 숫자 + 특수문자 

    if m:                                          # 특수 문자있는 경우 뒤에 절삭 
        start, end = m.span()
        x = x[:start+1]
#         x = x.replace(x[start+1:end],'') 
        return x
    
    else:
        return x

    
def weight_reg_02(x):
    
    if x.__contains__("미"):  # 미확인 & 미상 처리 
        return 'NaN' 
    
    else:
        x = x.rstrip('.')
        return x 

df['weight'] = df.weight.map(weight_reg_01)
df['weight'] = df.weight.map(weight_reg_02)

In [213]:
# weight_float 새로운 변수로 생성해둠 

weight_list = []

for i in list(df.weight):
    try:
        weight_list.append(float(i))
    except:
        weight_list.append(np.nan)
        
weight_list.count(np.nan)
df['weight_float'] = weight_list

#  5. 질적 변수 : 데이터 범주화 categorizing/factoring

## 상태 범주화 - 입양, 안락사

In [214]:
df.processstate.value_counts()

종료(자연사)    98836
종료(입양)     95112
종료(안락사)    66169
종료(반환)     35481
보호중        18044
종료(기증)      5733
종료(방사)      4161
종료(미포획)     1166
Name: processstate, dtype: int64

In [215]:
# 상태 범주화 : 입양 1, 나머지 0
def processstate_to_int(processstate):
    
    if processstate =='종료(입양)':
        return 1
    else: 
        return 0
    
df['processstate_adopt'] = df.processstate.map(processstate_to_int)
df.processstate_adopt = df.processstate_adopt.fillna('0')
df.processstate_adopt.value_counts()

0    229590
1     95112
Name: processstate_adopt, dtype: int64

In [216]:
# 상태 범주화 : 안락사 칼럼 생성
# 안락사 1, 나머지 0

def processstate_to_int(processstate):
    
    if processstate =='종료(안락사)':
        return 1
    else: 
        return 0
    
df['processstate_euthanize'] = df.processstate.map(processstate_to_int)
df.processstate_euthanize = df.processstate_euthanize.fillna('0')
df.processstate_euthanize.value_counts()

0    258533
1     66169
Name: processstate_euthanize, dtype: int64

## 성별 범주화

In [217]:
# 성별 범주화 
def gender_to_int(gender):

    if gender=='F':
        gender = 1
    elif gender=='M':
        gender = 0
    else:
        gender = 2
    return gender

df['sexcd'] = df.sexcd.map(gender_to_int)

In [218]:
# 중성화 여부
def neuteryn_to_int(neuteryn):

    if neuteryn=='N':
        neuteryn = 0
    elif neuteryn =='Y':
        neuteryn = 1
    else:
        neuteryn = 2
    return neuteryn

df['neuteryn'] = df.neuteryn.map(neuteryn_to_int)

In [219]:
# 유기동물 타입 : 1 = dog, 0 = cat
df['type'] = df.type.apply(lambda x : '1' if x == 'dog' else '0')

## 색상 범주화

In [220]:
# 색깔 파생변수 생성 
df['blc_col'] = 100
df['brown_col'] = 100
df['white_col'] = 100
df['grey_col'] = 100
df['yellow_col'] = 100

In [221]:
# 특수문자 제거 
def cleanse(x):
    x = x.strip()
    x = x.replace(".", "")
    x = x.replace("/", "")
    x = x.replace("+", "")
    x = x.replace(",", "")
    x = x.replace("&", "")
    x = x.replace("-", "")
    x = x.replace(" ", "")
    x = x.replace("(", "")
    x = x.replace(")", "")
    x = x.replace(";", "")
    return x

df['colorcd'] = df.colorcd.map(cleanse)

In [222]:
color_unique = df.colorcd.unique()
clist = color_unique.tolist()
clist

['검흰',
 '황색',
 '백색',
 '흰색',
 '회색',
 '흰갈',
 '검갈',
 '흰갈검',
 '검정',
 '흰색검은색',
 '검은색갈색',
 '노랑흰색',
 '흰색검정',
 '노랑',
 '노랑검정',
 '검정색',
 '갈색',
 '누런색',
 '흰황색',
 '은황색',
 '흰갈색',
 '흰검정',
 '흰색갈검색',
 '갈색검정',
 '흰',
 '검갈흰',
 '갈흰',
 '갈흰검',
 '연갈색',
 '흰색갈색',
 '검정황색',
 '흰갈회',
 '황',
 '갈색흰색',
 '호반색',
 '비글잡종갈색',
 '갈검흰',
 '갈검색',
 '아이보리',
 '파티',
 '화이트',
 '회갈색',
 '흰색살구색',
 '연갈색크림색흰색갈색',
 '살구색크림색',
 '연갈색크림색흰색',
 '연갈색크림색',
 '검정색갈색흰색',
 '검정색갈색',
 '검정색흰색',
 '갈색검정색흰색',
 '흰색갈색얼룩',
 '크림색갈색얼룩흰색검은얼룩',
 '흰색에검은얼룩',
 '황색얼룩',
 '흰색과흑색혼합',
 '크림색',
 '황흑백',
 '흑황백',
 '황백',
 '황백흑',
 '흑백',
 '아이보리백',
 '얼룩흰색검정',
 '연한갈색',
 '검정가슴에흰색',
 '베이지색',
 '흰검색',
 '검갈색',
 '흰색바탕에입주변검정',
 '백석',
 '흰검',
 '갈검',
 '흑갈색',
 '검은색',
 '검정갈색',
 '검정흰색',
 '검백색',
 '검정붉은갈색',
 '흰색베이지색',
 '얼룩',
 '갈흑',
 '회백색',
 '살구색',
 '백',
 '기타',
 '옅은갈색',
 '블랙앤화이트',
 '흰색검정색',
 '갈회',
 '밤색',
 '갈백색',
 '백황',
 '백검',
 '진갈색',
 '브린들',
 '점박이색',
 '연갈',
 '블랙',
 '블랙브라운',
 '흰색검은점박이',
 '갈',
 '검정색황색',
 '갈색검정색',
 '검정갈색흰색',
 '검정=갈색흰색',
 '쥐갈색',
 '쥐색',
 '검갈블랙탄',
 '흰색크림색',
 '흰색갈색검정색',
 '흰색검정갈색',
 '골든',


In [223]:
# 검정색

def blc_tf(x):

        if x.find('검') + x.find('흑') + x.find('블랙') + x.find('black') + x.find('Black') + x.find('BLACK') + x.find('건정') + x.find('겁정') + x.find('까') + x.find('껌') + x.find('달마시안') + x.find('바둑') + x.find('불랙') + x.find('검졍') + x.find('젖소') + x.find('ㄱ머정') + x.find('감정') + x.find('거멍') + x.find('거멎ㅇ') + x.find('거정') + x.find('걸정') + x.find('블렉') + x.find('젓소') + x.find('깜') + x.find('블랜') + x.find('호피') + x.find('컴정') + x.find('훅색') > -28:
            return 1
        else: 
            return 0

In [227]:
# 흰색

def white_tf(x):
        if x.find('흰') + x.find('백') + x.find('하') + x.find('벡') + x.find('크림') + x.find('아이보리') + x.find('희') + x.find('흐니') + x.find('휜') + x.find('white') + x.find('White') + x.find('WHITE') + x.find('힌') + x.find('크김') + x.find('화이트') + x.find('흐ㅟ') + x.find('달마시안') + x.find('바둑') + x.find('하얀') + x.find('환색') + x.find('횐') + x.find('흔색') + x.find('흼') + x.find('읜색') + x.find('읜') + x.find('젖소') + x.find('크리') + x.find('ㅎㄴ;섹') + x.find('하얀색') + x.find('흐ㅏㄴ') + x.find('희') + x.find('Ivory') + x.find('ivory') + x.find('젓소') + x.find('ㅎㄴ섹') + x.find('하얀색') + x.find('상아') + x.find('이아보리') + x.find('햐얀') + x.find('후ㅏㄴ샥') + x.find('힁') > -41:
            return 1
        else: 
            return 0

In [229]:
# 회색

def grey_tf(x):
    
        if x.find('쥐') + x.find('회') + x.find('은색') + x.find('은회') + x.find('실버') + x.find('재색') + x.find('그레이') + x.find('잿') + x.find('제색') + x.find('차콜') + x.find('grey') + x.find('gray') + x.find('먹색') + x.find('홰색') + x.find('화색') + x.find('화섹') > -16 and x.find('검은색') == -1:
            return 1
        else: 
            return 0
        if x.find[0] == '은':
            return 1

In [230]:
# 갈색

def brown_tf(x):

        if x.find('갈') + x.find('밤') + x.find('브라운') + x.find('초코') + x.find('고동') + x.find('흑갈') + x.find('베이지') + x.find('갈흑') + x.find('연갈') + x.find('쵸코') + x.find('초콜') + x.find('황토') + x.find('배이지') + x.find('길색') + x.find('apricot') + x.find('Apricot') + x.find('rkftor') + x.find('SaltPepper') + x.find('강객') + x.find('강색') + x.find('걀색') + x.find('걸샥') + x.find('길섹') + x.find('에프리코트') + x.find('에프리푸들') + x.find('연베이') + x.find('커피') + x.find('흙') + x.find('Tan') + x.find('tan') + x.find('brown') + x.find('Brown') + x.find('BROWN') + x.find('초쿄') + x.find('칡') + x.find('코코아') + x.find('가랙') + x.find('간색') + x.find('감색') + x.find('애프리') + x.find('애프리콧') + x.find('에프리') + x.find('고등어') + x.find('고긍어') + x.find('고둥어') + x.find('고드엉') + x.find('고등') + x.find('고등러') + x.find('고등색') + x.find('고릉어') + x.find('공등어') + x.find('탄') + x.find('호피') + x.find('호구') + x.find('재구') + x.find('카카오') > -56:
            return 1
        else: 
            return 0

In [231]:
# 노랑색

def yellow_tf(x):

        if x.find('황') + x.find('노랑') + x.find('골든') + x.find('옐') + x.find('금') + x.find('골') + x.find('누렁') + x.find('노란') + x.find('누') + x.find('치즈') + x.find('GOLD') + x.find('노') + x.find('모래') + x.find('yellow') + x.find('Yellow') + x.find('YELLOW') + x.find('ㅊㅣ즈') + x.find('치츠') + x.find('치느') + x.find('츠지') + x.find('gold') + x.find('ㄴ랑') + x.find('호구') + x.find('재구') > -24:
            return 1
        else: 
            return 0

In [232]:
# 함수적용

df['blc_col'] = df.colorcd.map(blc_tf)
df['white_col'] = df.colorcd.map(white_tf)
df['grey_col'] = df.colorcd.map(grey_tf)
df['brown_col'] = df.colorcd.map(brown_tf)
df['yellow_col'] = df.colorcd.map(yellow_tf)

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324702 entries, 0 to 324701
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   desertionno             324702 non-null  int64         
 1   happendt                324702 non-null  datetime64[ns]
 2   happenplace             324702 non-null  object        
 3   kindcd                  324702 non-null  object        
 4   colorcd                 324702 non-null  object        
 5   age                     324644 non-null  object        
 6   weight                  324702 non-null  object        
 7   noticeno                324701 non-null  object        
 8   noticesdt               324702 non-null  datetime64[ns]
 9   noticeedt               324702 non-null  datetime64[ns]
 10  sexcd                   324702 non-null  int64         
 11  neuteryn                324702 non-null  int64         
 12  specialmark             324702

## 품종 범주화

In [234]:
# 원본 데이터 <-> 품종코드 데이터 컬럼명 동일하게 수정
df = df.rename({'kindcd':'knm'}, axis = 'columns')

# dog/cat 분류 
dog = df.iloc[:212618]
cat = df.iloc[212618:]

# 개/ 고양이 품종 데이터 import 
dog_kind = pd.read_csv('./dog_kind')
cat_kind = pd.read_csv('./cat_kind')

dog_kind.drop('Unnamed: 0', axis = 1, inplace = True)
cat_kind.drop('Unnamed: 0', axis = 1, inplace = True)

In [235]:
# 전처리 : 개/고양이 각 테이블의 [개]/[고양이] 문자열 strip (변수에 담아서 확인)
dog['knm'] = dog['knm'].str.lstrip('[개] ')
cat['knm'] = cat['knm'].str.lstrip('[고양이] ')

In [236]:
# 중복값 확인
dog_kind.nunique()

knm       176
kindcd    177
dtype: int64

In [237]:
dog_kind.knm.value_counts()

도고 아르젠티노        2
이탈리안 그레이 하운드    1
시츄              1
비글              1
폭스테리어           1
               ..
토이 맨체스터 테리어     1
에어델 테리어         1
스코티시 디어하운드      1
바센지             1
그레이트 피레니즈       1
Name: knm, Length: 176, dtype: int64

In [238]:
dog_kind[dog_kind['knm'] == '도고 아르젠티노']

,knm,kindcd
14,도고 아르젠티노,42
15,도고 아르젠티노,153


In [239]:
#  중복값 처리 : 도고 아르젠티노 kindcd 153번 삭제
dog_kind.drop(15, axis = 0, inplace = True )

In [240]:
# [고양이]로만 기재된 데이터는 믹스로 추정하여 코드부여
cat_kind.loc[36] = ['', 212]

# '페르시안페르시안 친칠라' 코드 페르시안페르시안-친칠라와 동일하게 부여
cat_kind.loc[37] = ['페르시안페르시안 친칠라', 197]

In [241]:
## 공백 제거
dog['knm'].str.strip()
cat['knm'].str.strip()

212618          
212619          
212620          
212621          
212622          
           ...  
324697    한국 고양이
324698    한국 고양이
324699    한국 고양이
324700    한국 고양이
324701    한국 고양이
Name: knm, Length: 112084, dtype: object

In [242]:
# 특수 문자 제거

def cleanText(dataframe):
 
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', dataframe)
 
    return text

dog['knm'] = dog.knm.map(cleanText)
cat['knm'] = cat.knm.map(cleanText)

In [248]:
# knm을 key로 merge 진행
dog = pd.merge(dog, dog_kind, how='left', left_on='knm', right_on='knm')
cat = pd.merge(cat, cat_kind, how='left', left_on='knm', right_on='knm')

In [249]:
dog.kindcd.value_counts()

114.0    136960
72.0      13157
128.0     13093
67.0       7807
89.0       4412
          ...  
210.0         1
64.0          1
208.0         1
31.0          1
66.0          1
Name: kindcd, Length: 158, dtype: int64

In [250]:
# merge후 kindcd null값 확인
dog.kindcd.isnull().sum() # 2595

2595

In [251]:
# merge후 kindcd null값 확인
cat.kindcd.isnull().sum() # 4299

4299

In [252]:
# null 값인 행 눈으로 확인
dog[dog['kindcd'].isnull() == True]

,desertionno,happendt,happenplace,knm,colorcd,age,weight,noticeno,noticesdt,noticeedt,...,noticesdt_day,weight_float,processstate_adopt,processstate_euthanize,blc_col,brown_col,white_col,grey_col,yellow_col,kindcd
161,446498202000303,2020-10-13,황룡면 필암리,리트리버,갈색,2017(년생),8,전남-장성-2020-00303,2020-10-13,2020-10-23,...,13,8.00,0,0,0,1,0,0,0,NaN
284,442419202001046,2020-10-12,대안리,발바리,갈색,2018(년생),3.2,강원-원주-2020-00865,2020-10-14,2020-10-26,...,14,3.20,0,0,0,1,0,0,0,NaN
290,426330202000450,2020-10-12,명안로 26번길 12-3,혼합,검정갈색,2020(년생),2.1,부산-동래-2020-00358,2020-10-14,2020-10-26,...,14,2.10,0,0,1,1,0,0,0,NaN
291,426330202000449,2020-10-12,충렬대로 428번가길 19 남흥아파트 101동,혼합,흰색갈색,2018(년생),3.5,부산-동래-2020-00357,2020-10-14,2020-10-26,...,14,3.50,0,0,0,1,1,0,0,NaN
395,448546202000273,2020-10-12,함양군 마천면 금계안길,믹스,흰갈,2019(년생),3.6,경남-함양-2020-00121,2020-10-12,2020-10-22,...,12,3.60,0,0,0,1,1,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212138,450650201704410,2017-10-30,제주시 교래4길 10,허스키,흰검,2016(년생),20,제주-제주-2017-03840,2017-10-31,2017-11-10,...,31,20.00,1,0,1,0,1,0,0,NaN
212184,441405201701223,2017-10-30,처인구청,시츄믹스추정,연한갈색,2014(년생),3.85,경기-용인-2017-00711,2017-11-03,2017-11-13,...,3,3.85,0,0,0,1,0,0,0,NaN
212257,441405201701221,2017-10-31,양지cc주변,푸들믹스,흰색,2015(년생),6.5,경기-용인-2017-00709,2017-10-31,2017-11-12,...,31,6.50,0,0,0,0,1,0,0,NaN
212363,442424201700174,2017-10-31,마달동 혜진아파트 5동 계단,발바리,검정믹스,2015(년생),5,강원-삼척-2017-00179,2017-11-01,2017-11-13,...,1,5.00,0,0,1,0,0,0,0,NaN


In [255]:
# null 대부분이 knm이 '믹스'관련인 것 파악 
# 믹스관련 강아지에 품종 코드 부여: mapping 방식 적용할 수 있는 함수 dog_mix --> 훨씬 빠름!
def dog_mix(x):
    
    if (x.find('믹스견') + x.find('mix') + x.find('잡종') +
       x.find('믹스') + x.find('혼종') + x.find('혼합')+
       x.find('+') + x.find('추정') + x.find('?')+
       x.find('발바리')  + x.find('기타') + x.find('믹')+ x.find('49856') + x.find('잡견')) > -14:
        return '믹스견'

    else:
        return x

In [256]:
def huski_dog(x):
    
    if x.find('허스') > -1:
        return '시베리안 허스키'

    else:
        return x

In [257]:
# 함수 mapping
dog['knm'] = dog.knm.map(dog_mix)
dog['knm'] = dog.knm.map(huski_dog)

In [258]:
# 믹스관련 고양이에 품종 코드 부여: mapping 방식 적용할 수 있는 함수 cat_mix --> 훨씬 빠름!
def cat_mix(x):
    
    if (x.find('믹스묘') + x.find('mix') + x.find('잡종') +
       x.find('믹스') + x.find('혼종') + x.find('혼합')+
       x.find('+') + x.find('추정') + x.find('?')+
       x.find('발바리')  + x.find('기타') + x.find('믹') + x.find('54638')) > -13:
        return '믹스묘'

    else:
        return x

In [259]:
# 품종 '한국 고양이' 전처리
def korea_cat(x):
    
    if (x.find('코숏') + x.find('한국')) > -2:
        return '한국 고양이'

    else:
        return x

In [260]:
# 함수 mapping 
cat['knm'] = cat.knm.map(cat_mix)
cat['knm'] = cat.knm.map(korea_cat)

In [261]:
# 품종 전처리 후 다시 merge - 코드 부여
dog = pd.merge(dog, dog_kind, how='left', left_on='knm', right_on='knm')
cat = pd.merge(cat, cat_kind, how='left', left_on='knm', right_on='knm')

In [262]:
# merge하면서 생긴 필요없는 컬럼 drop
dog.drop('kindcd_x', axis = 1, inplace = True)
cat.drop('kindcd_x', axis = 1, inplace = True)

# 컬럼명 재지정
dog = dog.rename({'kindcd_y':'kindcd'}, axis = 'columns')
cat = cat.rename({'kindcd_y':'kindcd'}, axis = 'columns')

# null kindcd는 9999로 처리
dog.kindcd = dog.kindcd.fillna(np.nan)
dog.kindcd.astype('float')

cat.kindcd = cat.kindcd.fillna(np.nan)
cat.kindcd.astype('float')

0         212.0
1         212.0
2         212.0
3         212.0
4         212.0
          ...  
112079    200.0
112080    200.0
112081    200.0
112082    200.0
112083    200.0
Name: kindcd, Length: 112084, dtype: float64

In [263]:
dog_cat = pd.concat([dog, cat])

In [264]:
df = dog_cat

In [265]:
df.head()

,desertionno,happendt,happenplace,knm,colorcd,age,weight,noticeno,noticesdt,noticeedt,...,noticesdt_day,weight_float,processstate_adopt,processstate_euthanize,blc_col,brown_col,white_col,grey_col,yellow_col,kindcd
0,448535202001458,2020-10-13,김해시 주촌면 서부로1563번길 54,믹스견,검흰,2016(년생),8,경남-김해-2020-01466,2020-10-15,2020-10-26,...,15,8.0,0,0,1,0,1,0,0,114.0
1,446500202000046,2020-10-13,군내면 송산길 118,믹스견,황색,2019(년생),7,전남-진도-2020-00046,2020-10-15,2020-10-22,...,15,7.0,0,0,0,0,0,0,1,114.0
2,446500202000045,2020-10-13,군내면 송산길 118,믹스견,백색,2020(년생),10,전남-진도-2020-00045,2020-10-15,2020-10-22,...,15,10.0,0,0,0,0,1,0,0,114.0
3,446500202000044,2020-10-13,고군면 가계길 26-10,믹스견,황색,2019(년생),15,전남-진도-2020-00044,2020-10-15,2020-10-22,...,15,15.0,0,0,0,0,0,0,1,114.0
4,446499202000215,2020-10-13,금일 월송리 마을야산,믹스견,흰색,2017(년생),14,전남-완도-2020-00218,2020-10-15,2020-10-24,...,15,14.0,0,0,0,0,1,0,0,114.0
